文本预处理之文本特征处理
===
- 文本特征处理包括为语料添加具有普适性的文本特征，如n-gram特征，以及对加入特征在后的文本语料进行必要的处理，如长度规范。这些特征处理工作能够有效的将重要的文本特征加入模型训练中，增强模型评估指标

# 1.添加N-gram特征
给定一段文本序列，其中n个词或字的相邻共现特征即n-gram特征，常用的n-gram特征是bigram和tri-gram特征，分别对应n为2和3

## 1.1.定义
语言模型用来判断是否一句话从语法上是通顺的，具体办法就是求解整句话的概率。有一个句子s，其中s是由很多单词构成的:$\omega_1,\omega_2,...,\omega_n$，我们需要计算在这个语言模型下这个句子的概率，即

$$p(s)=p(\omega_1,\omega_2,...,\omega_n)$$

比如有一句话"全民AI是趋势"，首先经过分词，得到"全民，AI，是，趋势"这些词，那么就有
$$P_{LM}(全民AI是趋势)=P_{LM}(全民，AI，是，趋势)$$

## 1.2.计算方法-Chain Rule

$$P(A,B,C,D)=P(A) \times P(B|A) \times P(C|A,B) \times P(D|A,B,C)$$
$$P(\omega_1,\omega_2,...,\omega_n)=P(\omega_1) \times P(\omega_2|\omega_1) \times ... \times P(\omega_n|\omega_1,\omega_2,...,\omega_{n-1})$$

举例来说，$P(地位|京东，AI，技术，处于，领先)$，它的意思是，在语料库中，出现京东AI技术处于领先这句话的同时，后面一个词是"地位"的概率，比如我们的语料库中有两个地方出现了"京东AI技术处于领先"，其中一个地方的后面的词是“地位”，另一个后面的词是“的”，那么它的概率就是$\frac{1}{2}$。但是“京东AI技术处于领先”这句话太长了，很有可能在语料库中根本就没出现过，那么概率就是0了，这就导致了稀疏性。于是我们就会使用马尔科夫的假设

## 1.3.马尔科夫假设与N-gram
$P(地位|京东，AI，技术，处于，领先)$

### 1.3.1.阶马尔科夫假设-Unigram Model(1-gram Model)
假定上述概率约等于$P(地位)$

### 1.3.2.阶马尔科夫假设-Bigram Model(2-gram Model)
假定上述概率约等于$P(地位|京东)$

### 1.3.3.阶马尔科夫假设-Trigram Model(3-gram Model)
假定上述概率约等于$P(地位|京东，AI)$

对于$P(\omega_1,\omega_2,...\omega_n)$来说，如果使用1-gram Model,就变成了
$$P(\omega_1) \times P(\omega_2) \times ... \times P(\omega_n)=\prod_{i=1}^nP(\omega_i)$$
如果使用2-gram Model，就变成了
$$P(\omega_1) \times P(\omega_2|\omega_1) \times ... \times P(\omega_n|\omega_{n-1})=P(\omega_1)\prod_{i=2}^nP(\omega_i|\omega_{i-1})$$
如果使用3-gram Model，就变成了
$$P(\omega_1) \times P(\omega_2|\omega_1) \times ... \times P(\omega_n|\omega_{n-2},\omega_{n-1})=P(\omega_1) \times P(\omega_2|\omega_1)\prod_{i=3}^nP(\omega_i|\omega_{i-2},\omega_{i-1})$$

我们一般使用2-gram Model

## 1.4.举例

### 1.4.1.实例一
假设有概率如下

| 语料 | 概率 |
| --- | ---- |
| p(是\|今天) | 0.01 |
| p(今天) | 0.002 |
| p(周日\|是) | 0.001 |
| p(周日\|今天) | 0.0001 |
| p(周日) | 0.02 |
| p(是\|周日) | 0.0002 |

比较：今天是周日 VS 今天周日是

$$P(今天，是，周日)=P_{LM}(今天) \times P_{LM}(是|今天) \times P_{LM}(周日|是)=0.002 \times 0.01 \times 0.001 = 2 \times 10^{-8}$$
$$P(今天，周日，是)=P_{LM}(今天) \times P_{LM}(周日|今天)) \times P_{LM}(是|周日)=0.002 \times 0.0001 \times 0.0002 = 4 \times 10^{-11}$$
所以第一句话更符合语法

# 2.文本长度规范